This code (will be) set up to generate a forecasted impact map of a specified area.
The area in the example case is Houston, TX
The inundation extent is sourced from the National Water Model forecast. 
Depth model is calculated from the inundation extent and the national 3m DEM
The impact map is calculated from the depth and the CDC SVI scores at the census tract scale. 

In [ ]:
# edit code so that it runs automatically every hour
# use python.12.4 kernel
# THIS IS A WORK IN PROGRESS

Currently, the dependencies require numpy <2, so make sure your numpy package matches that requirement. If it does, skip the next two steps

In [1]:
import os
import numpy as np
import arcpy
from datetime import datetime

## Import packages and set the paths to the working directory and input files
TO DO: 
1. Replace the directory path with the path to the folder you want to work in
2. Replace dem_name with path to the dem you want to use (as a tif) 
3. Replace the inund_polygon with path to the area extent you want to map (as a .shp)

In [5]:
# Import packages
import os
import arcpy
from datetime import datetime

# Format the current datetime as a string in the desired format (e.g., YYYYMMDD_HHMMSS)
datetime = datetime.now().strftime("%Y%m%d_%H%M%S")


# Replace 'path/to/your/directory' with the path to your desired working directory
directory = 'C:/Users/Lyn/Documents/ArcGIS/CUAHSI_SI_2024/07122024_5day'


# Create a new folder within the current working directory
resultfolder = "5dayforecast_07092024"
os.makedirs(os.path.join(directory, resultfolder), exist_ok=True)
print(f"New folder created at: {os.path.join(directory, resultfolder)}")
infolder = directory +'/'+ resultfolder + '/'


# set up input variables
dem_name = 'C:/Users/Lyn/Documents/Programming/Python_codes/fwdet/Forecast_test_5day/Houston_DEM.tif'
extent = 'C:/Users/Lyn/Documents/ArcGIS/CUAHSI_SI_2024/Houston_huc.shp'
svi = 'C:/Users/Lyn/Documents/Programming/Python_codes/Impact_Map_Forecasting/Texas_example/aligned_svi_raster.tif'

# Set the working directory
os.chdir(infolder)
# Verify the current working directory
print("Current Working Directory: ", os.getcwd())

# set up input and output variables
arcpy.env.overwriteOutput = True

# make a geodatabase within the infolder   
gdb_path = infolder 
# Create the geodatabase
arcpy.CreateFileGDB_management(out_folder_path = infolder, out_name='geodatabase.gdb')
#workspace
ws = arcpy.env.workspace = infolder

New folder created at: C:/Users/Lyn/Documents/ArcGIS/CUAHSI_SI_2024/07122024_5day\5dayforecast_07092024
Current Working Directory:  C:\Users\Lyn\Documents\ArcGIS\CUAHSI_SI_2024\07122024_5day\5dayforecast_07092024


In [ ]:
%pip install geopandas rasterio shapely

## Query the NWM 5 day inundation exent. This section will download it as a shapefile, merge polygons, and prepare for depth calculaiton

In [6]:
# query the NOAA API for the most recent flood predictions
import requests
import geopandas as gpd

# Construct the query URL
# To change the query, you can go to https://maps.water.noaa.gov/server/rest/services/nwm
# Then select the link you want, copy it here, and add /0/query to the end
# e.g. https://link_to_service/0/query
query_url = "https://maps.water.noaa.gov/server/rest/services/nwm/mrf_gfs_5day_max_inundation_extent/MapServer/0/query"
params = {
    'where': '1=1',
    'outFields': '*',
    'outSR': '4326',  # Specify output spatial reference if needed
    'f': 'geojson',  # Request geojson output
    'returnGeometry': 'true'
}
# Make the request
response = requests.get(query_url, params=params)


# Check if the server is running correctly. A response of 500 means there is a server error. 
if response.status_code == 200:
    try:
        # Attempt to convert the response to a GeoDataFrame
        predictions = gpd.GeoDataFrame.from_features(response.json(), crs="EPSG:4326")
        print(predictions.head())
    except ValueError as e:
        # Handle JSON decoding errors
        print(f"Error decoding JSON: {e}")
else:
    # The request failed; print the status code and response text
    print(f"Request failed with status {response.status_code}: {response.text}")



# Convert to GeoDataFrame
predictions = gpd.GeoDataFrame.from_features(response.json(), crs="EPSG:4326")
print(predictions.head())


                                            geometry feature_id  \
0  MULTIPOLYGON (((-99.06275 27.50207, -99.06276 ...     322529   
1  MULTIPOLYGON (((-99.10026 27.52774, -99.10016 ...     322547   
2  POLYGON ((-99.09559 27.52107, -99.09568 27.520...     322547   
3  MULTIPOLYGON (((-99.0996 27.50099, -99.09949 2...     322547   
4  MULTIPOLYGON (((-99.07352 27.50175, -99.07342 ...     322551   

   streamflow_cfs           reference_time              update_time  oid  
0           24.37  2024-07-12 12:00:00 UTC  2024-07-12 18:30:32 UTC    1  
1            4.94  2024-07-12 12:00:00 UTC  2024-07-12 18:30:32 UTC    2  
2            4.94  2024-07-12 12:00:00 UTC  2024-07-12 18:30:32 UTC    3  
3            4.94  2024-07-12 12:00:00 UTC  2024-07-12 18:30:32 UTC    4  
4           21.90  2024-07-12 12:00:00 UTC  2024-07-12 18:30:32 UTC    5  
                                            geometry feature_id  \
0  MULTIPOLYGON (((-99.06275 27.50207, -99.06276 ...     322529   
1  MULTIPOLYG

In [17]:
# To use if you already have a fim file that is cleaned up
import geopandas as gpd

# Specify the path to the shapefile
shapefile_path = "F:/SI_2024/5day_07182024_smooth.shp"

# Read the shapefile into a GeoDataFrame
predictions = gpd.read_file(shapefile_path)

# Print the GeoDataFrame
print(predictions.head())

# To change the CRS to WGS84 (EPSG:4326) as an example
gdf = predictions.to_crs(epsg=4326)
predictions = gdf


   COUNT    SHAPE_Leng    SHAPE_Area  \
0  924.0  8.358015e+06  3.540893e+08   

                                            geometry  
0  MULTIPOLYGON (((-10666025.504 3471520.993, -10...  


### Clip the predictions to the huc extent

In [10]:
# Clip predictions to huc extent

import geopandas as gpd

# Replace 'path/to/your/shapefile.shp' with the actual path to your shapefile
shapefile_path = extent
huc = gpd.read_file(shapefile_path)

# To change the CRS to WGS84 (EPSG:4326) as an example
gdf = huc.to_crs(epsg=4326)
huc = gdf
# Now `gdf` is a GeoDataFrame containing the data from your shapefile

# clip the predictions to the extent of interest - defined by the dem_name above

import geopandas as gpd
import rasterio
from shapely.geometry import box

# Assuming `gdf_to_clip` is your GeoDataFrame that you want to clip,
# and `clipping_polygon` is a GeoDataFrame containing the polygon to clip to.

# Step 1: Ensure both GeoDataFrames are in the same CRS
if predictions.crs != huc.crs:
    gdf_to_clip = predictions.to_crs(huc.crs)
else:
    gdf_to_clip = predictions
# Step 2: Use the clip function
clipped_gdf = gpd.clip(gdf_to_clip, huc)

# `clipped_gdf` now contains the geometries from `gdf_to_clip` clipped to the extent of `clipping_polygon`.
predictions = clipped_gdf

In [11]:
# If predictions is empty, print a message and exit the script
import sys
if predictions.empty:
    print("No flood predictions in this area found.")
    sys.exit(0)

### Clean up the shapefiles a bit

In [17]:
# Dissolve all polygons
dissolved_gdf = predictions.dissolve()
predictions = dissolved_gdf


In [18]:
# Specify the path where you want to save the shapefile
output_path = infolder + 'test.shp'

# Export the dissolved GeoDataFrame as a shapefile
dissolved_gdf.to_file(output_path)

C:\Users\Lyn\AppData\Local\Temp\ipykernel_7140\3429588930.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  dissolved_gdf.to_file(output_path)
C:\Users\Lyn\AppData\Roaming\Python\Python311\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'streamflow_cfs' to 'streamflow'
  ogr_write(
C:\Users\Lyn\AppData\Roaming\Python\Python311\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'reference_time' to 'reference_'
  ogr_write(
C:\Users\Lyn\AppData\Roaming\Python\Python311\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'update_time' to 'update_tim'
  ogr_write(


## FWDET2.1 calculates depth map from inundation extent
### Written by Dr.Sagy Cohen

In [24]:
"""
Calculate water depth from a flood extent polygon (e.g. from remote sensing analysis) based on
an underlying DEM (or HAND).
Program procedure:
    1. Flood extent polygon to polyline
    2. Polyline to Raster - DEM extent and resolution (Env)
    3. Con - DEM values to Raster
    4. Euclidean Allocation - assign boundary cell elevation to nearest domain cells
    5. Calculate water depth by deducting DEM by Euclidean Allocation
    6. Run low-pass Filter
Created by Sagy Cohen, Surface Dynamics Modeling Lab, University of Alabama
email: sagy.cohen@ua.edu
web: http://sdml.ua.edu

NOTE: the shapefile must be smaller than the dem extent
"""


import arcpy
from arcpy.sa import *
# Checkout the Spatial Analyst Toolkit
arcpy.CheckOutExtension(['Spatial'])
arcpy.env.overwriteOutput = True

def main():
    ####################
    #   INPUT/OUTPUT   #
    ####################
    # Inputs#
    ws = arcpy.env.workspace = infolder
    inund_polygon = output_path

    clip_dem = '' #[Optional] - If empty, the clip_dem will be calculated with the Clip_management function
    cost_raster ='' #[Optional] - If empty, the CostRaster will be calculated below

    #Parameters#
    numIterations = 10 #number of smoothing iterations
    slopeTH = 0.5 #filtering slope threshold

    #Output# water depth raster
    Out_WaterDepth = 'WaterDepth'

    # Create raster object from DEM
    dem = arcpy.Raster(dem_name)
    # Check if optional Cost Raster was provided
    if not cost_raster:
        cost_raster = (((dem <= 0)*999)+1)
        cost_raster.save(ws + '\CostRaster')

    # Cell size here would be the x or y distance resolution from the raster
    # i.e., a 30 meter dem would have a cell size of 30
    cell_size = dem.meanCellHeight
    # Proper string representation of dem extent to be accepted by Clip_management method
    extent = '{} {} {} {}'.format(dem.extent.XMin, dem.extent.YMin, dem.extent.XMax, dem.extent.YMax)
    
    # If optional clip dem no provided then create a clipping dem cut out from the flood inundation polygon
    if not clip_dem:
        clip_dem = 'ClipDEM'
        arcpy.management.Clip(dem, extent, clip_dem, inund_polygon, nodata_value= "-9999", clipping_geometry="ClippingGeometry", maintain_clipping_extent="NO_MAINTAIN_EXTENT")
    clip_dem_ras = arcpy.Raster(clip_dem)
    # Generate raster line. See CalculateBoundary docstring for more info
    boundary = CalculateBoundary(dem, clip_dem_ras, inund_polygon, cell_size, numIterations, slopeTH)
    arcpy.AddMessage('Calculated Boundary')

    #   arcpy.env.extent = arcpy.Extent(dem.extent.XMin, dem.extent.YMin, dem.extent.XMax, dem.extent.YMax)
    # Convert boundary, i.e., raster line to int for cost allocation function. It only takes int rasters
    MULTIPLIER = 10000
    boundary_int = Int(boundary * MULTIPLIER)
    boundary_int.save("boundary_int")
    arcpy.AddMessage('Running cost allocation')
    with arcpy.EnvManager(snapRaster=None, extent="DEFAULT", mask=clip_dem):
        # cost_alloc = CostAllocation(boundary, cost_raster, '#', '#', 'Value')
        cost_alloc = CostAllocation(boundary_int, cost_raster, '#', '#', 'Value')
    
    # Divide the result from the cost allocation function using the same constant used to create the integer
    # representation of the boundary
    cost_alloc = Float(cost_alloc) / MULTIPLIER
    arcpy.AddMessage('Cost Allocation raster generated')
    arcpy.AddMessage('Calculating estimated water depth')
    # Raster calculator cost_alloc - clip_dem
    water_depth = (cost_alloc - clip_dem_ras)
    
    # Remove estimated water depths below 0 and change them to 0
    #water_depth = Con(water_depth <= 0, 0, water_depth)
    water_depth = Con(water_depth > 0, water_depth,"#")
    water_depth.save(Out_WaterDepth)
    arcpy.AddMessage('Floodwater depth computed')
    #Run a low-pass filter
    arcpy.AddMessage('Running low-pass Filter')
    water_depth_filtered = Filter(water_depth, 'LOW', 'DATA')
    waterDepthFilter2 = Con(clip_dem_ras, water_depth_filtered, '#', 'VALUE > 0')
    waterDepthFilter2.save(Out_WaterDepth+'_filtered')
    
def CalculateBoundary(dem, clip_dem_ras, inund_polygon, cell_size,numIterations,slopeTH):
    """
    Return a raster line representation with associated underlying DEM values as the values.
    Take in a inundated flood polygon, create a polyline representation of the input inundation_polygon.
    Next, convert flood polygon polyline calculated in the first step to a raster.
    Then, set values of newly created 'raster line' to the underlying dem values.
    Finally, save the raster line to the workspace
    Much of the naming conventions found in this function follow the arcpy documentation for the 'Con' function.
    Input:
        dem -> ArcPy raster object
        inundation_polygon -> str
        cell_size -> int
    Return:
        str of raster line
    """
    arcpy.AddMessage('Converting inundation polygon to inundation polyline')
    # Convert inundation extent polygon to polyline
    polyline = 'polyline'
    arcpy.PolygonToLine_management(inund_polygon, polyline)
    arcpy.AddMessage('Converting inundation polyline to raster')
    # Convert polyline to raster
    with arcpy.EnvManager(snapRaster=clip_dem_ras):
        arcpy.conversion.PolylineToRaster(polyline, 'FID', 'linerast15', "MAXIMUM_LENGTH", "NONE", cell_size)
    raster_polyline = Raster('linerast15')
    try:
    # Assuming raster_polyline is created somewhere above this
        if raster_polyline is not None:
            raster_polyline.save("polyline.tif")
        else:
            print("raster_polyline is None or invalid.")
    except Exception as e:
        print(f"An error occurred while saving the raster polyline: {e}")
    # The input whose values will be used as the output cell values if the condition is false.
    inFalseConstant = '#'
    where_clause = 'VALUE >= 0'
    #Extract the boundary cells elevation from DEM
    boundary = Con(raster_polyline, dem, inFalseConstant, where_clause)
    # boundary.save('boundary1')
    #Smooth boundary raster
    iterations = int(numIterations)
    for i in range(iterations):
        arcpy.AddMessage('Focal iteration '+str(i+1))
        OutRasTemp = FocalStatistics(boundary, "Rectangle 5 5 CELL", 'MEAN', 'DATA')
        boundary = Con(raster_polyline, OutRasTemp, inFalseConstant, where_clause)
        boundary.save('boundary'+str(i+1))
    #Identify and remove ocean boundary cells
    OutRasTemp = FocalStatistics(dem, 'Circle 2 CELL', 'MINIMUM', 'DATA') 
    whereClause2 = 'VALUE > 0'
    boundary = Con(OutRasTemp, boundary, inFalseConstant, whereClause2)
    boundary.save("boundaryAfterOcean")
    if slopeTH>0.0:
    #calculate topo slope
        arcpy.AddMessage('Calculating Slope')
        extent_clip = '{} {} {} {}'.format(boundary.extent.XMin, boundary.extent.YMin, boundary.extent.XMax, boundary.extent.YMax)
        with arcpy.EnvManager(extent=extent_clip):
            out_slope = arcpy.sa.Slope(dem, "PERCENT_RISE", 1, "GEODESIC", "METER")
            out_slope.save("Slope_m")
    #Remove erroneous boundary cells 
        whereClause_slope = 'VALUE > ' + str(slopeTH)
        #boundary = arcpy.sa.Con(out_slope, boundary, None, "VALUE > 1.0")
        boundary = Con(out_slope, boundary, inFalseConstant, whereClause_slope)
  
    boundary.save("boundFinal")
    # Removing created eronious generated objects
   # arcpy.Delete_management(raster_polyline)
   # arcpy.Delete_management(polyline)
    return boundary
main()


RuntimeError: Invalid pointer 

In [20]:

#clip_dem = '' #[Optional] - If empty, the clip_dem will be calculated with the Clip_management function
#cost_raster ='' #[Optional] - If empty, the CostRaster will be calculated below
WaterDepthOutput = 'WaterDepth_i10_s0p5'
iterations = 10
SlopeFiltering = True
SlopeThreshold = 0.5
inund_polygon = predictions # This is a new addition!! is it correct??? is inund_polgon supposed to be the predictions?

In [21]:
# clean and create necessary variables
dem = arcpy.Raster(dem_name)
cell_size = dem.meanCellHeight
# Proper string representation of dem extent to be accepted by Clip_management method
extent = '{} {} {} {}'.format(dem.extent.XMin, dem.extent.YMin, dem.extent.XMax, dem.extent.YMax)

# Dont run these if cost_raster and clip_dem are already defined above
cost_raster = (((dem <= 0)*999)+1)
cost_raster.save('CostRaster.tif')

clip_dem = 'ClipDEM'


In [22]:
# clip the DEM to the huc extent
import geopandas as gpd
import rasterio
from rasterio.mask import mask

# Step 2: Load the HUC shapefile
huc_shapefile_path = extent
huc_gdf = huc

# Step 3: Read the DEM file
dem_path = dem_name
with rasterio.open(dem_path) as dem_src:
    # Step 4: Convert HUC geometry to DEM CRS, if necessary
    if huc_gdf.crs != dem_src.crs:
        huc_gdf = huc_gdf.to_crs(dem_src.crs)
    
    # Get the geometry in GeoJSON format
    huc_geom = [huc_gdf.geometry.unary_union.__geo_interface__]
    
    # Step 5: Clip the DEM
    out_image, out_transform = mask(dem_src, huc_geom, crop=True)
    
    # Define metadata for the new file
    out_meta = dem_src.meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})
    
    # Step 6: Save the clipped DEM
    with rasterio.open('clipped_dem.tif', 'w', **out_meta) as out_file: ### ??? edit the clipped_dem.tif to store in the way we want
        out_file.write(out_image)

### ??? how to make dem from above cell equal the output of this cell??

C:\Users\Lyn\AppData\Local\Temp\ipykernel_7140\2676260078.py:18: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  huc_geom = [huc_gdf.geometry.unary_union.__geo_interface__]


In [23]:
# Calculate Boundary Raster
arcpy.env.overwriteOutput = True
polyline = infolder + '/polyline.shp'
arcpy.PolygonToLine_management(inund_polygon, polyline)
# Convert polyline to raster
with arcpy.EnvManager(snapRaster=clip_dem_ras):
    arcpy.conversion.PolylineToRaster(polyline, 'FID', 'linerast15', "MAXIMUM_LENGTH", "NONE", cell_size)
raster_polyline = arcpy.Raster('linerast15')
raster_polyline.save("rstr_poly")
# The input whose values will be used as the output cell values if the condition is false.
inFalseConstant = '#'
where_clause = 'VALUE >= 0'
#Extract the boundary cells elevation from DEM
boundary = arcpy.sa.Con(raster_polyline, dem, inFalseConstant, where_clause)
# boundary.save('boundary1')
#Smooth boundary raster
for i in range(iterations):
    OutRasTemp = arcpy.sa.FocalStatistics(boundary, "Rectangle 5 5 CELL", 'MEAN', 'DATA')
    boundary = arcpy.sa.Con(raster_polyline, OutRasTemp, inFalseConstant, where_clause)
    boundary.save('boundary'+str(i+1))
#Identify and remove ocean boundary cells
OutRasTemp = arcpy.sa.FocalStatistics(dem, 'Circle 2 CELL', 'MINIMUM', 'DATA') 
whereClause2 = 'VALUE > 0'
boundary = arcpy.sa.Con(OutRasTemp, boundary, inFalseConstant, whereClause2)
#boundary.save("boundaryAfterOcean")
if SlopeFiltering:
#calculate topo slope
    print('Calculating Slope')
    extent_clip = '{} {} {} {}'.format(boundary.extent.XMin, boundary.extent.YMin, boundary.extent.XMax, boundary.extent.YMax)
    with arcpy.EnvManager(extent=extent_clip):
        out_slope = arcpy.sa.Slope(dem, "PERCENT_RISE", 1, "GEODESIC", "METER")
        out_slope.save("Slope_m")
#Remove erroneous boundary cells 
    whereClause_slope = 'VALUE > ' + str(SlopeThreshold)
    boundary = arcpy.sa.Con(out_slope, boundary, inFalseConstant, whereClause_slope)
boundary.save("boundFinal")
print('Finished calculating the boundary raster')

RuntimeError: Object: Error in executing tool

In [ ]:
# Calculate water depth 

MULTIPLIER = 10000
boundary_int = arcpy.sa.Int(boundary * MULTIPLIER)
#boundary_int.save("boundary_int")
print('Running cost allocation')
with arcpy.EnvManager(snapRaster=None, extent="DEFAULT", mask=clip_dem):
    cost_alloc = arcpy.sa.CostAllocation(boundary_int, cost_raster, '#', '#', 'Value')

# Divide the result from the cost allocation function using the same constant used to create the integer
# representation of the boundary
cost_alloc = arcpy.sa.Float(cost_alloc) / MULTIPLIER
print('Cost Allocation raster generated')
print('Calculating estimated water depth')
water_depth = (cost_alloc - clip_dem_ras)
# Remove estimated water depths below 0 and change them to 0
water_depth = arcpy.sa.Con(water_depth > 0, water_depth,"#")

In [ ]:
# Run a low-pass filter
# ie smooth the water depth raster
print('Running low-pass Filter')
water_depth_filtered = arcpy.sa.Filter(water_depth, 'LOW', 'DATA')
waterDepthFilter2 = arcpy.sa.Con(clip_dem_ras, water_depth_filtered, '#', 'VALUE > 0')
#waterDepthFilter2.save(WaterDepthOutput+'_filtered')
print('Finished low-pass Filter calculation')
waterDepthFilter2

In [ ]:
# Export the waterdepth as a tif file
# Set the output file path
waterdepth_path = infolder + 'waterDepth.tif'

# Export the waterDepthFilter2 raster as a TIFF file
waterDepthFilter2.save(waterdepth_path)

# Print the output file path
print(f"Water depth filter exported as: {waterdepth_path}")

## Run SVI using depth map

#### Step 1: Load the ESRI Geodatabase and Clip the Data

In [ ]:
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as colors
import matplotlib.colorbar as colorbar
import rasterio
from rasterio.features import rasterize
from rasterio.plot import show
import folium
from rasterio.warp import calculate_default_transform, reproject, Resampling
from matplotlib.colors import Normalize
from rasterio.transform import from_bounds
from osgeo import gdal, ogr, osr
import os

# Caculate the Impact 

#### Step 1: read the svi raster and fim raster
If you have not have an svi map yet, you need to run through Flood Impact Map.ipynb workflow to produce one. Use the output of that here. 

In [ ]:
import rasterio

# Define the paths to the raster files - same as the output above
svi_raster_path = svi
fim_raster_path = waterdepth_path

def check_raster_properties(raster_path):
    with rasterio.open(raster_path) as src:
        print(f"Properties of {raster_path}:")
        print(f" CRS: {src.crs}")
        print(f" Cell size: {src.res}")
        print(f" Width: {src.width}, Height: {src.height}")
        print(f" Bounds: {src.bounds}")
        print()

# Read and check properties of SVI raster
check_raster_properties(svi_raster_path)

# Read and check properties of FIM raster
check_raster_properties(fim_raster_path)


#### Step 2: Reproject and Resample SVI Raster
Skip this if they have same projection and cell size

##### make sure that the svi map has the same crs and cell size of fim map

In [ ]:
import os
from osgeo import gdal
import rasterio

# Define the paths to the raster files
aligned_svi_raster_path = infolder + 'aligned_svi_raster2.tif'

# Open the FIM raster to get the projection, geotransform, and size
fim_ds = gdal.Open(fim_raster_path)
if fim_ds is None:
    raise RuntimeError(f"Failed to open FIM raster {fim_raster_path}")

fim_proj = fim_ds.GetProjection()
fim_geotransform = fim_ds.GetGeoTransform()
fim_width = fim_ds.RasterXSize
fim_height = fim_ds.RasterYSize

print(f"FIM Raster Properties:\n CRS: {fim_proj}\n Geotransform: {fim_geotransform}\n Width: {fim_width}, Height: {fim_height}")

# Reproject and resample the SVI raster to match the FIM raster
warp_options = gdal.WarpOptions(
    format='GTiff',
    outputBounds=(fim_geotransform[0], fim_geotransform[3] + fim_height * fim_geotransform[5], fim_geotransform[0] + fim_width * fim_geotransform[1], fim_geotransform[3]),
    width=fim_width,
    height=fim_height,
    dstSRS=fim_proj,
    xRes=fim_geotransform[1],
    yRes=-fim_geotransform[5],
    resampleAlg=gdal.GRA_Bilinear,
    targetAlignedPixels=True
)

print("Warp options set.")

aligned_svi_raster = gdal.Warp(aligned_svi_raster_path, svi_raster_path, options=warp_options)
if aligned_svi_raster is None:
    raise RuntimeError(f"gdal.Warp failed to create {aligned_svi_raster_path}")

print("SVI raster has been reprojected and resampled to match the FIM raster.")

# Verify the properties of the aligned SVI raster
def check_raster_properties(raster_path):
    with rasterio.open(raster_path) as src:
        print(f"Properties of {raster_path}:")
        print(f" CRS: {src.crs}")
        print(f" Cell size: {src.res}")
        print(f" Width: {src.width}, Height: {src.height}")
        print(f" Bounds: {src.bounds}")
        print()

# Check properties of the aligned SVI raster
check_raster_properties(aligned_svi_raster_path)


Reclassify fim

In [ ]:
import numpy as np
import rasterio

def reclassify_fim(input_path, output_path):
    with rasterio.open(input_path) as src:
        data = src.read(1).astype('float32')
        nodata = src.nodata

        if nodata is not None:
            data = np.ma.masked_equal(data, nodata)
        
        # Reclassify the FIM data
        reclassified_data = np.digitize(data, bins=[0.01, 0.5, 1, 2], right=True)
        reclassified_data = np.where(data == 0, 0, reclassified_data)  # Class 0 for non-flooded areas

        # Preserve nodata values
        reclassified_data = np.ma.filled(reclassified_data, nodata)

        meta = src.meta
        meta.update(dtype='float32', nodata=-9999)

        with rasterio.open(output_path, 'w', **meta) as dst:
            dst.write(reclassified_data.astype('float32'), 1)


reclassified_fim_path = infolder + 'aligned_fim.tif'

reclassify_fim(fim_raster_path, reclassified_fim_path)
print("Reclassified FIM raster has been saved.")


Create impact map

In [ ]:
def multiply_rasters(svi_path, fim_path, output_path):
    with rasterio.open(svi_path) as svi_src:
        svi_data = svi_src.read(1).astype('float32')
        svi_nodata = svi_src.nodata

    with rasterio.open(fim_path) as fim_src:
        fim_data = fim_src.read(1).astype('float32')
        fim_meta = fim_src.meta
        fim_nodata = fim_src.nodata

    if svi_data.shape != fim_data.shape:
        raise ValueError("The rasters do not have the same shape.")

    # Ensure that nodata values are handled correctly
    nodata_value = -9999
    svi_data = np.where(svi_data == svi_nodata, np.nan, svi_data)
    fim_data = np.where(fim_data == fim_nodata, np.nan, fim_data)

    # Multiply the rasters to get the impact map, ignoring NaN values
    impact_data = np.where(np.isnan(svi_data) | np.isnan(fim_data), nodata_value, svi_data * fim_data)

    # Save the impact map to a new raster file
    impact_meta = fim_meta.copy()
    impact_meta.update({
        'dtype': 'float32',
        'nodata': nodata_value
    })

    with rasterio.open(output_path, 'w', **impact_meta) as impact_dst:
        impact_dst.write(impact_data.astype('float32'), 1)

svi_raster_path = svi
impact_raster_path = infolder + 'impact_raster.tif'

multiply_rasters(svi_raster_path, reclassified_fim_path, impact_raster_path)
print("Impact map has been saved.")


Mask 0s in the impact map

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

def mask_zeros(input_path, output_path):
    with rasterio.open(input_path) as src:
        data = src.read(1).astype('float32')
        nodata = src.nodata

        masked_data = np.where(data == 0, nodata, data)

        meta = src.meta
        meta.update(dtype='float32', nodata=nodata)

        with rasterio.open(output_path, 'w', **meta) as dst:
            dst.write(masked_data.astype('float32'), 1)


masked_impact_raster_path = infolder + 'masked_impact_raster.tif'

mask_zeros(impact_raster_path, masked_impact_raster_path)
print("Masked impact map has been saved.")

def plot_raster(ax, raster_path, title):
    with rasterio.open(raster_path) as src:
        data = src.read(1)
        nodata = src.nodata
        if nodata is not None:
            data = np.ma.masked_equal(data, nodata)  # Mask NoData values

        img = ax.imshow(data, cmap='plasma', norm=Normalize(vmin=data.min(), vmax=data.max()))
        ax.set_title(title)
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        return img

# Create a figure with subplots
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the masked impact raster
img = plot_raster(ax, masked_impact_raster_path, "Masked Impact Raster")

# Add colorbars with size control
cbar = fig.colorbar(img, ax=ax, fraction=0.046, pad=0.04, shrink=0.4)
cbar.set_label('Impact Value')

# Adjust layout
plt.tight_layout()
plt.show()


Reclassify the impact map

In [ ]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

def classify_raster(input_path, output_path):
    with rasterio.open(input_path) as src:
        data = src.read(1).astype('float32')
        nodata = src.nodata

        # Mask nodata values
        data_masked = np.ma.masked_equal(data, nodata)
        
        # Identify the range of the data excluding nodata values
        min_value = data_masked.min()
        max_value = data_masked.max()
        print(f"Data range: {min_value} to {max_value}")

        # Classify the data into 4 equal intervals
        bins = np.linspace(min_value, max_value, 5)  # 5 bins will create 4 intervals
        classified_data = np.digitize(data_masked, bins, right=True)  # Digitize creates 1 to 4 classes

        # Convert the classified_data mask to nodata values
        classified_data[data_masked.mask] = nodata

        print(f"Classified data range: {np.min(classified_data)} to {np.max(classified_data)}")

        meta = src.meta
        meta.update(dtype='float32', nodata=nodata)

        with rasterio.open(output_path, 'w', **meta) as dst:
            dst.write(classified_data.astype('float32'), 1)


classified_impact_raster_path = infolder + 'classified_impact_raster.tif'

classify_raster(masked_impact_raster_path, classified_impact_raster_path)
print("Classified impact map has been saved.")

def plot_raster(ax, raster_path, title):
    with rasterio.open(raster_path) as src:
        data = src.read(1)
        nodata = src.nodata
        if nodata is not None:
            data = np.ma.masked_equal(data, nodata)  # Mask NoData values

        img = ax.imshow(data, cmap='plasma', norm=Normalize(vmin=1, vmax=4))  # Set vmin and vmax to match class values
        ax.set_title(title)
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        return img

# Create a figure with subplots
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the classified impact raster
img = plot_raster(ax, classified_impact_raster_path, "Classified Impact Raster")

# Add colorbars with size control
cbar = fig.colorbar(img, ax=ax, fraction=0.046, pad=0.04, shrink=0.4)
cbar.set_label('Impact Class')

# Adjust layout
plt.tight_layout()
plt.show()


Make shapefile

In [ ]:
from osgeo import gdal, ogr
import numpy as np


def raster_to_shapefile(raster_path, shapefile_path):
    # Open the reclassified raster
    src_ds = gdal.Open(raster_path)
    if src_ds is None:
        raise RuntimeError(f"Failed to open raster {raster_path}")
    
    # Get the raster band
    src_band = src_ds.GetRasterBand(1)
    nodata = src_band.GetNoDataValue()
    
    # Create the output shapefile
    driver = ogr.GetDriverByName('ESRI Shapefile')
    if driver is None:
        raise RuntimeError("ESRI Shapefile driver not available.")
    
    dst_ds = driver.CreateDataSource(shapefile_path)
    if dst_ds is None:
        raise RuntimeError(f"Failed to create shapefile {shapefile_path}")
    
    srs = ogr.osr.SpatialReference()
    srs.ImportFromWkt(src_ds.GetProjection())
    
    dst_layer = dst_ds.CreateLayer('impact', srs=srs, geom_type=ogr.wkbPolygon)
    if dst_layer is None:
        raise RuntimeError(f"Failed to create layer in shapefile {shapefile_path}")
    
    field_defn = ogr.FieldDefn('FIV_Class', ogr.OFTInteger)
    dst_layer.CreateField(field_defn)
    
    # Polygonize the raster, skipping NoData values
    gdal.Polygonize(src_band, src_band.GetMaskBand(), dst_layer, 0, options=["8CONNECTED=8"], callback=None)
    
    # Remove features with the NoData value
    dst_layer.SetAttributeFilter(f"FIV_Class = {int(nodata)}")
    for feature in dst_layer:
        dst_layer.DeleteFeature(feature.GetFID())
    dst_layer.SetAttributeFilter(None)
    
    # Close the datasets
    src_ds = None
    dst_ds = None

# Path to the reclassified impact raster file
reclassified_raster_path = classified_impact_raster_path
shapefile_path = infolder + 'reclassified_impact_raster.shp'


# Convert the reclassified raster to a shapefile
raster_to_shapefile(reclassified_raster_path, shapefile_path)
print("Shapefile has been created.")


add column to attribute table of FIV map to describe the severity of the vulnerability

In [ ]:
from osgeo import ogr

def add_vulnerability_column(shapefile_path):
    # Open the shapefile
    driver = ogr.GetDriverByName('ESRI Shapefile')
    ds = driver.Open(shapefile_path, 1)  # Open in write mode
    if ds is None:
        raise RuntimeError(f"Failed to open shapefile {shapefile_path}")
    
    layer = ds.GetLayer()
    
    # Add new field for vulnerability levels
    vulnerability_field = ogr.FieldDefn('Vuln_Level', ogr.OFTString)
    vulnerability_field.SetWidth(32)
    layer.CreateField(vulnerability_field)
    
    # Define the vulnerability levels
    vulnerability_levels = {
        1: "Low Vulnerability",
        2: "Medium Vulnerability",
        3: "High Vulnerability",
        4: "Very High Vulnerability"    
    }
    
    # Update the new field based on FIV_Class
    for feature in layer:
        fiv_class = feature.GetField('FIV_Class')
        vulnerability_level = vulnerability_levels.get(fiv_class, "Unknown")
        feature.SetField('Vuln_Level', vulnerability_level)
        layer.SetFeature(feature)
    
    # Close the dataset
    ds = None
    print("Vulnerability level column has been added to the shapefile.")

# Path to the shapefile
shapefile_path = infolder+'ImpactMap.shp'

# Add the vulnerability level column
add_vulnerability_column(shapefile_path)


# Export outputs to ArcGIS online

This is the notebook to update the arcgis instant app
This requires arcpy as a package
Before running this notebook, you should have already set up the acrGIS instant app. This notebook only replaces the shapefile already in the map, it does not create a new map. You need to do that first

## Raster to Shapefile conversion

In [ ]:
# raster to shapefile conversion

from osgeo import gdal, ogr
import numpy as np


def raster_to_shapefile(raster_path, shapefile_path):
    # Open the reclassified raster
    src_ds = gdal.Open(raster_path)
    if src_ds is None:
        raise RuntimeError(f"Failed to open raster {raster_path}")
    
    # Get the raster band
    src_band = src_ds.GetRasterBand(1)
    nodata = src_band.GetNoDataValue()
    
    # Create the output shapefile
    driver = ogr.GetDriverByName('ESRI Shapefile')
    if driver is None:
        raise RuntimeError("ESRI Shapefile driver not available.")
    
    dst_ds = driver.CreateDataSource(shapefile_path)
    if dst_ds is None:
        raise RuntimeError(f"Failed to create shapefile {shapefile_path}")
    
    srs = ogr.osr.SpatialReference()
    srs.ImportFromWkt(src_ds.GetProjection())
    
    dst_layer = dst_ds.CreateLayer('impact', srs=srs, geom_type=ogr.wkbPolygon)
    if dst_layer is None:
        raise RuntimeError(f"Failed to create layer in shapefile {shapefile_path}")
    
    field_defn = ogr.FieldDefn('FIV_Class', ogr.OFTInteger)
    dst_layer.CreateField(field_defn)
    
    # Polygonize the raster, skipping NoData values
    gdal.Polygonize(src_band, src_band.GetMaskBand(), dst_layer, 0, options=["8CONNECTED=8"], callback=None)
    
    # Remove features with the NoData value
    dst_layer.SetAttributeFilter(f"FIV_Class = {int(nodata)}")
    for feature in dst_layer:
        dst_layer.DeleteFeature(feature.GetFID())
    dst_layer.SetAttributeFilter(None)
    
    # Close the datasets
    src_ds = None
    dst_ds = None

# Path to the reclassified impact raster file
reclassified_raster_path = 'F:/SI_2024/Final__inputs_outputs/Final__inputs_outputs/Original_reclassified_Depths_fwdet/reclassified_fwdet_Beryl_18hr_08Jul_08_10_nonad.tif'

# the output needs to be in its own folder
# Define the path for the new folder
shapefile_path = 'F:/SI_2024/Final__inputs_outputs/Final__inputs_outputs/18hr_depth_shapefile/depthmap'
# Create the new folder
os.makedirs(shapefile_path, exist_ok=True)
print(f"New folder created at: {shapefile_path}")

# Convert the reclassified raster to a shapefile
raster_to_shapefile(reclassified_raster_path, shapefile_path)
print("Shapefile has been created.")


In [ ]:
from arcgis.gis import GIS
import os

### zip the shapefile

In [ ]:
# zip the shapefile
import shutil
shapefile_name = 'reclassified_depth.shp'

# Create a zip file of the source folder
shutil.make_archive(shapefile_name, 'zip', shapefile_path)


To create the GIS object, we pass your profile that contains the url and your login credentials. Please replace the credentials below with that of your org. 
To learn more about profiles, see [here](https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/#Storing-your-credentialls-locally).

In [ ]:
gis = GIS('home')


#If this does not automatically connect, you may need to do this code instead:
#gis = GIS(
#  url="https://www.arcgis.com",
#  username="username",
#  password=getpass.getpass("Enter password:")
#)

# or do this:
# gis = GIS("https://www.arcgis.com", "your_username", "your_password")

### Delete the former shapefile so we can upload the new one

In [ ]:
# search for the layer id
search_result = gis.content.search('title:FIM', item_type = 'Feature Service')
display(search_result)

In [ ]:
# Replace 'item_id' with the actual ID of the item you want to delete
item_id = search_result[0].id
item_to_delete = gis.content.get(item_id)

# Delete the item, remove it permanently is necesary if you have a recycle bin enabled at your organization
item_to_delete.delete(permanent=True)

In [ ]:
# search for the shapefile ID
search_result = gis.content.search('title:FIM', item_type = 'Shapefile')
display(search_result)

# Replace 'item_id' with the actual ID of the item you want to delete
item_id = search_result[0].id
item_to_delete = gis.content.get(item_id)

# Delete the item, remove it permanently is necesary if you have a recycle bin enabled at your organization
item_to_delete.delete(permanent=True)


## upload the new shapefile to ArcGIS Online
edit the metadata with the title and tags of the file. the path must lead to a zip file

In [ ]:
# get datetime to tag the file name
from datetime import datetime

current_datetime = datetime.now()
print(current_datetime)

# Format the datetime as a string (e.g., "2023-04-01 12:00")
formatted_datetime = current_datetime.strftime('%Y-%m-%d %H:%M')

In [ ]:
# edit the dictionary with the title and tags you want
metadata = {
    'title': f'FIM {formatted_datetime}',
    'tags': 'Flood Forecast, 5 Days, Inundation, National Water Model',
    'type': 'Shapefile'
}
data_path = os.path.join('C:/Users/Lyn/Desktop/Berylb.zip')

In [ ]:
shapefile = gis.content.add(metadata, data=data_path)
feature_layer = shapefile.publish()
feature_layer.url

##  Add the layer to the webmap Instant App

In [ ]:
from arcgis.mapping import WebMap, WebScene
from IPython.display import display
import json


# ID the web map you want to alter
search_result = gis.content.search("title:Flood Forecast Houston", item_type = "Web Map")

# read the webmap as an object
wm_item = search_result[0]
web_map_obj = WebMap(wm_item)

In [ ]:
# check the layers on the web map
for lyr in web_map_obj.layers:
    print(lyr.title + " " + lyr.url)

In [ ]:
# Feature service item id for the missing layer:
web_map_obj.layers[0]["itemId"]

to do : edit the title to match the one above

In [ ]:

# search for the newly uploaded layer
search_result = gis.content.search('title:Berylb', item_type = 'Feature Service')
display(search_result)

# check the item id
search_result[0].id

# update the web map with the new layer
capitals = search_result[0]
capitals.layers

In [ ]:
# The new feature service does have a layer with id 0. Hence we can use the same layer id while switching the url. 
# To remove the old layer, call remove_layer() method. 
# Then add the correct FeatureLayer object by calling the add_layer() method on the WebMap object.

# remove the old layer from the web map
web_map_obj.remove_layer(web_map_obj.layers[0])

# add the correct layer. While adding you can customize the title
web_map_obj.add_layer(capitals.layers[0], options={'title':'Berylb'})

#check the layers on the web map
for lyr in web_map_obj.layers:
    print(lyr.title + " " + lyr.url)

# update the web map 
web_map_obj.update(item_properties={'title':'USA Capitals - updated'})



to do: Make sure what you delete matches the variable names above

In [ ]:
# Let's clean it up so we can always run this section again
wm_item.delete()
#new_capitals.delete()
#csv_item.delete()